![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 2 Notebook - AutoAI Notebook v1.15.2

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a> 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.7 and scikit-learn 0.23.2.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn.


In [ ]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs==1.12.9 | tail -n 1
!pip install -U scikit-learn==0.23.2 | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import S3Connection, S3Location

training_data_reference = [
    DataConnection(
    connection=S3Connection(
        api_key='kmS1UYEzyjcBnZuyO2VMN8vYchXlEo652Bmb7Eyx8T7D',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3.eu.cloud-object-storage.appdomain.cloud'
    ),
        location=S3Location(
            bucket='specwater-donotdelete-pr-s2nov85kwrxpuj',
            path='amostra_agua_oficial5.csv'
        )
    ),
]
training_result_reference = DataConnection(
    connection=S3Connection(
        api_key='kmS1UYEzyjcBnZuyO2VMN8vYchXlEo652Bmb7Eyx8T7D',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3.eu.cloud-object-storage.appdomain.cloud'
    ),
    location=S3Location(
        bucket='specwater-donotdelete-pr-s2nov85kwrxpuj',
        path='auto_ml/245884be-b067-48a4-9f81-3027d6850235/wml_data/3c30d88f-7903-4c12-986f-173d032bb9d3/data/automl',
        model_location='auto_ml/245884be-b067-48a4-9f81-3027d6850235/wml_data/3c30d88f-7903-4c12-986f-173d032bb9d3/data/automl/hpo_d_output/Pipeline1/model.pickle',
        training_status='auto_ml/245884be-b067-48a4-9f81-3027d6850235/wml_data/3c30d88f-7903-4c12-986f-173d032bb9d3/training-status.json'
    )
)

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='Target',
    holdout_size=0.1,
    scoring='accuracy',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_reference=training_data_reference,
    training_result_reference=training_result_reference,
    deployment_url='https://eu-de.ml.cloud.ibm.com',
    project_id='4e2a4cb3-b9da-41a0-8f97-2aef44780ae6',
    positive_label=1,
    drop_duplicates=True
)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.  
**Note**: If reading data using Flight Server for data connections results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [ ]:
df = training_data_reference[0].read(experiment_metadata=experiment_metadata)
df.dropna('rows', how='any', subset=[experiment_metadata['prediction_column']], inplace=True)

<a id="split"></a>
##  Train and test data split

In [ ]:
from sklearn.model_selection import train_test_split

df.drop_duplicates(inplace=True)
X = df.drop([experiment_metadata['prediction_column']], axis=1).values
y = df[experiment_metadata['prediction_column']].values

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=experiment_metadata['holdout_size'],
                                                    stratify=y, random_state=experiment_metadata['random_state'])

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from autoai_libs.transformers.exportable import float32_transform
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

#### Pre-processing & Estimator.

In [ ]:
numpy_column_selector = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
        71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
        88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
        160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
        174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
        188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
        202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,
        216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
        230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243,
        244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257,
        258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271,
        272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285,
        286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299,
    ]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
decision_tree_classifier = DecisionTreeClassifier(
    class_weight="balanced",
    max_depth=5,
    max_features=0.9492490250885458,
    random_state=33,
)


#### Pipeline.

In [ ]:
pipeline = make_pipeline(
    numpy_column_selector,
    float_str2_float,
    numpy_replace_missing_values,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
    decision_tree_classifier,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X,train_y)

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X, test_y)
print(score)

### Calling the `predict` method
If you want to get a prediction using pipeline model object, call `pipeline.predict()`.

**Note:** If you want to work with pure sklearn model:
 - add the following parameter to `get_pipeline` call: `astype='sklearn'`,
 - or `scikit_learn_pipeline = pipeline.export_to_sklearn_pipeline()`

In [ ]:
pipeline.predict(test_X)

<a id="next_steps"></a>
# Next steps

#### [Model deployment as webservice](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/deployments/autoai)
#### [Run AutoAI experiment with python SDK](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/experiments/autoai)  

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2021 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___